In [ ]:
!pip install rich-theme-manager
from rich.console import Console
from rich.style import Style
import pathlib
from rich_theme_manager import Theme, ThemeManager

THEMES = [
    Theme(
        name="dark",
        description="Dark mode theme",
        tags=["dark"],
        styles={
            "repr.own": Style(color="#e87d3e", bold=True),
            "repr.tag_name": "dim cyan",
            "repr.call": "bright_yellow",
            "repr.str": "bright_green",
            "repr.number": "bright_red",
            "repr.none": "dim white",
            "repr.attrib_name": Style(color="#e87d3e", bold=True),
            "repr.attrib_value": "bright_blue",
            "default": "bright_white on black"
        },
    ),
    Theme(
        name="light",
        description="Light mode theme",
        styles={
            "repr.own": Style(color="#22863a", bold=True),          #Class names
            "repr.tag_name": Style(color="#00bfff", bold=True),     #Adjust tag names
            "repr.call": Style(color="#ffff00", bold=True),         #Function calls and other symbols
            "repr.str": Style(color="#008080", bold=True),          #String representation
            "repr.number": Style(color="#ff6347", bold=True),       #Numbers
            "repr.none": Style(color="#808080", bold=True),         #None
            "repr.attrib_name": Style(color="#ffff00", bold=True),  #Attribute names
            "repr.attrib_value": Style(color="#008080", bold=True), #Attribute values
            "default": Style(color="#000000", bgcolor="#ffffff"),   #Default text and background
        },
    ),
]

theme_dir = pathlib.Path("themes").expanduser()
theme_dir.expanduser().mkdir(parents=True, exist_ok=True)

theme_manager = ThemeManager(theme_dir=theme_dir, themes=THEMES)
theme_manager.list_themes()

dark = theme_manager.get("dark")
theme_manager.preview_theme(dark)

In [ ]:
from rich.console import Console

dark = theme_manager.get("dark")
#Console with the dark theme
console = Console(theme=dark)

In [ ]:
import warnings

#Warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd

data = (
    pd.read_csv(
        '/content/sample_data/microbiologyevents.csv',
        on_bad_lines='skip',
        engine='python'
    )
    .reset_index(drop=True)
    .to_dict('records')
)
console.print(data[:2])

In [ ]:
!pip install qdrant-client sentence-transformers
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

#Creating the vector database client
qdrant = QdrantClient(":memory:")

#Creating the embedding encoder
encoder = SentenceTransformer('pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb') #Model to create embeddings

In [ ]:
#Creating collection to store the medical report data
collection_name="Medical_reports"

qdrant.recreate_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),
        distance=models.Distance.COSINE
    )
)

In [ ]:
#Vectorize!
qdrant.upload_points(
    collection_name=collection_name,
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["comments"]).tolist(),
            payload=doc
        ) for idx, doc in enumerate(data)
    ]
)

In [ ]:
console.print(qdrant.get_collection(collection_name=collection_name))

In [ ]:
user_prompt = "Give me the details of gram stain results that found polymorphonuclear leukocytes but no microorganisms"

In [ ]:
query_vector = encoder.encode(user_prompt).tolist()

In [ ]:
# hits = qdrant.query_points(
#     collection_name=collection_name,
#     query=query_vector,
#     limit=5
# )

In [ ]:
from rich.console import Console
from rich.text import Text
from rich.table import Table

console = Console()

table = Table(title="Retrieval Results", show_lines=True)

#Medical data columns
table.add_column("Subject ID", style="#e0e0e0")
table.add_column("Test Name", style="bright_red")
table.add_column("Comments", style="#89ddff")
table.add_column("Score", style="#a6accd")

results = qdrant.query_points(
    collection_name=collection_name,
    query=query_vector,
    limit=3
)

for hit in results.points:
    table.add_row(
        str(hit.payload.get("subject_id", "N/A")),
        hit.payload.get("test_name", "N/A"),
        f'{hit.payload.get("comments", "")[:100]}...',
        f"{hit.score:.4f}"
    )

console.print(table)

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from rich.panel import Panel
from rich.text import Text
from rich.console import Console

console = Console()

print("Loading TinyLlama...")
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
model = AutoModelForCausalLM.from_pretrained(
    "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    torch_dtype=torch.float16,
    device_map="auto"
)
print("Model loaded!")

#TinyLlama format
system_message = "You are a medical assistant. Your priority is to help analyze medical test results and provide clear, accurate information."
prompt = f"<|system|>\n{system_message}</s>\n<|user|>\n{user_prompt}</s>\n<|assistant|>\n"

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=300, temperature=0.7)

response_text_str = tokenizer.decode(outputs[0], skip_special_tokens=True)
response_text_str = response_text_str.split("<|assistant|>")[-1].strip()

styled_panel = Panel(
    Text(response_text_str),
    title="Report Analysis without Retrieval",
    expand=False,
    border_style="bold green",
    padding=(1, 1)
)

console.print(styled_panel)

In [ ]:
#Variable to hold the search results
search_results = [hit.payload for hit in results.points]

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from rich.panel import Panel
from rich.text import Text
from rich.console import Console

console = Console()

print("Loading TinyLlama...")
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
model = AutoModelForCausalLM.from_pretrained(
    "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    torch_dtype=torch.float16,
    device_map="auto"
)
print("Model loaded!")

#TinyLlama format
system_message = "You are a medical assistant. Your priority is to help analyze medical test results and provide clear, accurate information."
prompt = f"<|system|>\n{system_message}</s>\n<|user|>\n{user_prompt}</s>\n<|results|>\n{search_results}</s>\n"

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=300, temperature=0.7)

response_text_str = tokenizer.decode(outputs[0], skip_special_tokens=True)
response_text_str = response_text_str.split("<|assistant|>")[-1].strip()

styled_panel = Panel(
    Text(response_text_str),
    title="Report Analysis with Retrieval",
    expand=False,
    border_style="bold green",
    padding=(1, 1)
)

console.print(styled_panel)